In [23]:
import Robogame as rg
import networkx as nx
import time, json
import altair as alt
import pandas as pd
import numpy as np

# Simulator
This is an example of a simulator. It will create bets for the two players and the just see what happens.

I'm assuming two teams... one with the secret "alice" and one with "bob." 

To launch the server for this you can do the following in your server directory:

```python api.py -d ./example1 -t1s bob -t2s alice --matchsave bobvalice.json examplematch1```

This will launch the example1match from the example1 directory, using the two secrets (alice and bob). The log of the game will get saved to bobvalice.json which might be useful for analysis after the fact.

In [24]:
# let's read the real match data... this will give us the *real* answers
# you can modify this for different example files but remember to also launch the right server version
robotdata = pd.read_csv("../server/example1/examplematch1.robotdata.csv")

In [25]:
# let's create objects for our two teams
team1 = rg.Robogame("bob")
team2 = rg.Robogame("alice")

In [26]:
# tell the server we're ready
print(team1.setReady())
print(team2.setReady())

{'Error': 'Game already started'}
{'Error': "Team secret doesn't match any team"}


In [27]:
# wait for the match to launch
while(True):
    gametime = team1.getGameTime()
    timetogo = gametime['gamestarttime_secs'] - gametime['servertime_secs']
    if ('Error' in gametime):
        print("Error"+str(gametime))
        break
    if (timetogo <= 0):
        print("Let's go!")
        break
    
    print("waiting to launch... game will start in " + str(int(timetogo)))
    time.sleep(1) # sleep 1 second at a time, wait for the game to start

Let's go!


In [28]:
# both teams initially bet 50
omniplayerAllBets1 = {}
omniplayerAllBets2 = {}

for i in np.arange(0,100):
    omniplayerAllBets1[int(i)] = int(50)
    omniplayerAllBets2[int(i)] = int(50)
    
team1.setBets(omniplayerAllBets1)
team2.setBets(omniplayerAllBets2)

{'Error': "Team secret doesn't match any team"}

In [ ]:
# let's try to figure out some strategy for each robot based on the real data
# this runs a dumb simulation. It basically knows the truth but it will guess a random number
# around the truth that is proportional to the number of hints it knows about each robot
# the more hints it gets about a robot, the closer it will guess to the truth
#
# the two teams have slightly different amounts of random distributions so in theory
# team 2 will converge on the true answer faster than team 1

lasthints1 = {}
lasthints2 = {}

for timeloop in np.arange(0,100):
    # load some new hints
    team1.getHints()
    team2.getHints()
    
    omniplayerAllBets1 = {}
    omniplayerAllBets2 = {}

    x = team1.getRobotInfo()
    t = team1.getGameTime()
    if 'curtime' in t:
        if (t['curtime'] >= 100):
            break
    else:
        break
      
    # grab the robots that we can guess for
    
    
    # lets check all the hints we've seen so far
    t1hints = team1.getAllPredictionHints()
    t2hints = team2.getAllPredictionHints()

    hints1 = {}
    hints2 = {}
    
    # how many hints do we have per robot?
    for t1 in t1hints:
        id1 = int(t1['id'])
        if (id1 not in hints1):
            hints1[id1] = 1
        else:
            hints1[id1] = hints1[id1] + 1
        
    
    
    for t1 in t2hints:
        id1 = int(t1['id'])
        if (id1 not in hints2):
            hints2[id1] = 1
        else:
            hints2[id1] = hints2[id1] + 1
    
    #print(hints1,hints2)
    
    for row in robotdata[robotdata.id < 100].sort_values('id').iterrows():

        row = row[1]                # get the row data
        expires = int(row.expires)  # figure out when the robot expires
        col = "t_"+str(expires)     # look up the value at that time
        valAtExp = row[col]         # this is the true value
        
        if (expires <= t['curtime']):
            continue                   # skip expired robots

        rowid = int(row.id)

        if (rowid in hints1):
            if ((rowid not in lasthints1) or (hints1[rowid] != lasthints1[rowid])):
                # we have some evidence for this robot
                # pick a random number around the true answer (normal distrib with mean of valAtExp, stdev of 20)
                val = int(np.random.normal(valAtExp,20/hints1[rowid]))
                if (val <  0):
                    val = 0
                if (val > 100):
                    val = 100
                #print(row.id)
                omniplayerAllBets1[rowid] = val

        if (rowid in hints2):
             if ((rowid not in lasthints2) or (hints2[rowid] != lasthints2[rowid])):
                # pick a random number around the true answer (normal distrib with mean of valAtExp, stdev of 10)
                # this will be for team2 which are going to be slightly more accurate
                val = int(np.random.normal(valAtExp,10/hints2[rowid]))
                if (val <  0):
                    val = 0
                if (val > 100):
                    val = 100

                omniplayerAllBets2[rowid] = val

    lasthints1 = hints1
    lasthints2 = hints2
    print("team 1 bets",omniplayerAllBets1)
    print("team 2 bets",omniplayerAllBets2)
    team1.setBets(omniplayerAllBets1)
    team2.setBets(omniplayerAllBets2)
    print("time: ",t['curtime'])
    print(x[['winner','Productivity']].groupby(['winner']).sum())

    time.sleep(6)

In [ ]:
# that's more or less it... you can modify the guesses us different strategies
# for example, you can also consider the network

In [ ]:
omniplayerAllBets2